# Setup

Checking CUDA Installation



In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


Install Extension to Enable NVCC in Notebook Cells

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-hl8lzsm4
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-hl8lzsm4
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10739 sha256=99634096e9ae99b658cd96f87c6e8ccf9cbc16bf9f7ab36b8a182687000c7e83
  Stored in directory: /tmp/pip-ephem-wheel-cache-07qkwnal/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


Checking the installation success state


In [ ]:
!pip show nvcc4jupyter

Name: nvcc4jupyter
Version: 1.2.1
Summary: Jupyter notebook plugin to run CUDA C/C++ code
Home-page: 
Author: 
Author-email: Andrei Nechaev <lyfaradey@yahoo.com>, Cosmin Stefan Ciocan <ciocan.cosmin98@gmail.com>
License: MIT License
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [ ]:
!pip install gdown

# Download Test case and refs
!gdown --folder https://drive.google.com/drive/folders/1ayltedwtb5j6-VNyUeH-XHpJ-f9dGrGd?usp=sharing

Retrieving folder contents
Processing file 1uS0KIdpHINiR5be5MwqO9VXXqxdcFilZ 32.txt
Processing file 1lzqpgaxVCHE4WH81Yj0B0Fctk4ewp2P9 64.txt
Processing file 1lhMG8S4isg0ZKBvZV2-vuBOfF0wRWvWG 128.txt
Processing file 1wePn9jkHy0tbIxNM82VKE2xJ3-05sgU9 256.txt
Processing file 1ijPv8dz27cpjAn1MfcXEQ5DXNPD9Cjv5 512.txt
Processing file 1pbbjPMFNtAtuRW7SREjl79gBjHXIaF0Z 1024.txt
Processing file 1NlA9nepEwymBZeEf7-8-LQ2tsr0NHgs3 2048.txt
Processing file 1tR_OduEdQQCkRcdghCVKMaplVXHH2GM1 ser_32.txt
Processing file 1M4weCiu_iYGqYxQ5f9f2rNKA51OeQpdn ser_64.txt
Processing file 1RowyXjsR9f0JwkCKy6NhEs3PQ016F8nB ser_128.txt
Processing file 1ERfb8P4VpC599XFWVodenV3aTY2rxluI ser_256.txt
Processing file 1ojoV4_A64qdeuGqDXKWbJP_KIEyXhuJ- ser_512.txt
Processing file 15G6xVXf30C3kLZyYsyJgr9f9U-xKGGbR ser_1024.txt
Processing file 1Uy-kjtnQjeNwWxopfRYNjZ8DKmOlW5_Z ser_2048.txt
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https:/

# Kode CUDA

In [ ]:
%%writefile cuda_optimize.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#define EPSILON 1e-9

__global__ void normalize_row(double *in, double *out, int row, int N, int stride) {
    int _col = blockIdx.x * blockDim.x + threadIdx.x;
    int _row = blockIdx.y * blockDim.y + threadIdx.y;

    if (_row == row && _col < stride) {
        double pivot_value = in[row * stride + row];
        if (fabs(pivot_value) > EPSILON) {
            out[row * stride + _col] /= pivot_value;
        } else {
            printf("Pivot too small or zero at row %d, cannot normalize.\n", row);
        }
    }
}

__global__ void eliminate_row(double *in, double *out, int pivot_row, int N, int stride) {
    int _col = blockIdx.x * blockDim.x + threadIdx.x;
    int _row = blockIdx.y * blockDim.y + threadIdx.y;

    if (_row != pivot_row && _row < N && _col < stride) {
        // printf("_col %d; _row %d;\n", _col, _row);
        double factor = in[_row * stride + pivot_row];
        if (fabs(factor) > EPSILON) {
            out[_row * stride + _col] -= factor * in[pivot_row * stride + _col];
        }
    }
}

void read_matrix(double *matrix, int N)
{
    // Read the matrix from stdin
    for (int i = 0; i < N; ++i)
    {
        int curr_offset = i * 2 * N;
        for (int j = 0; j < N; ++j)
        {
            // printf("Reading element %d\n", i * 2 * N + j);
            if (scanf("%lf", &matrix[curr_offset + j]) != 1)
            {
                fprintf(stderr, "Error reading matrix element at position (%d, %d).\n", i, j);
                exit(1);
            }
        }
    }
}

void initialize_identity(double *matrix, int N) {
    for (int i = 0; i < N; i++) {
        for (int j = N; j < 2 * N; j++) {
            matrix[i * 2 * N + j] = (i == (j - N)) ? 1.0 : 0.0;
        }
    }
}

void print_matrix(double *matrix, int num_row, int num_col)
{
    // Print the matrix
    printf("Matrix:\n");
    for (int i = 0; i < num_row; ++i)
    {
        for (int j = 0; j < num_col; ++j)
        {
            // printf("\n%d, %d\n", i, j);
            printf("%lf ", matrix[i * num_col + j]);
        }
        printf("\n");
    }
}

void invert_matrix(double *h_matrix, int N) {
    double *d_matrix, *temp_matrix;
    int stride = 2 * N;

    cudaMalloc(&d_matrix, sizeof(double) * N * stride);
    cudaMemcpy(d_matrix, h_matrix, sizeof(double) * N * stride, cudaMemcpyHostToDevice);

    cudaMalloc(&temp_matrix, sizeof(double) * N * stride);
    cudaMemcpy(temp_matrix, h_matrix, sizeof(double) * N * stride, cudaMemcpyHostToDevice);

    dim3 blocks((stride + 15) / 16, (N + 15) / 16);
    dim3 threads(16, 16);

    for (int i = 0; i < N; i++) {
        // printf("START NORMALIZE ROW %d\n", i);
        normalize_row<<<blocks, threads>>>(d_matrix, temp_matrix, i, N, stride);
        cudaDeviceSynchronize();
        // printf("FINISH NORMALIZE ROW %d\n", i);
        cudaMemcpy(d_matrix, temp_matrix, sizeof(double) * N * stride, cudaMemcpyDeviceToDevice);

        // printf("START ELIMINATE ROW %d\n", i);
        eliminate_row<<<blocks, threads>>>(d_matrix, temp_matrix, i, N, stride);
        cudaDeviceSynchronize();
        // printf("FINISH ELIMINATE ROW %d\n", i);
        cudaMemcpy(d_matrix, temp_matrix, sizeof(double) * N * stride, cudaMemcpyDeviceToDevice);
    }

    cudaMemcpy(h_matrix, d_matrix, sizeof(double) * N * stride, cudaMemcpyDeviceToHost);
    // print_matrix(h_matrix, N, 2*N);
    cudaFree(d_matrix);
}

void save_file(double *matrix, int N)
{
    char filename[100]; // Adjust the size according to your needs
    snprintf(filename, sizeof(filename), "cuda_inverse_opt_%d.txt", N);

    // Open the file
    FILE *file = fopen(filename, "w");
    if (file != NULL)
    {
        fprintf(file, "%d\n", N);
        for (int i = 0; i < N; i++)
        {
            int curr_offset = i * 2 * N;
            for (int j = N; j < 2 * N; j++)
            {
                fprintf(file, "%f ", matrix[curr_offset + j]);
            }
            fprintf(file, "\n");
        }
        fclose(file);
        printf("Result matrix logged to file named cuda_inverse_opt_%d.txt.\n", N);
    }
    else
    {
        printf("Failed to open file for logging.\n");
    }
}

int main(void) {
    int N;
    double *matrix = NULL;

    if (scanf("%d\n", &N) == 1)
    {
        matrix = (double *)malloc(sizeof(double) * N * 2 * N);
        if (matrix == NULL)
        {
            fprintf(stderr, "Malloc error for matrix! Aborting.. \n");
            exit(1);
        }
        read_matrix(matrix, N);
        initialize_identity(matrix, N);
    }

    invert_matrix(matrix, N);

    save_file(matrix, N);

    free(matrix);
    return 0;
}

Writing cuda_optimize.cu


In [ ]:
!nvcc cuda_optimize.cu -o cuda_optimize

# Execution

In [ ]:
!time ./cuda_optimize < 2048.txt

Result matrix logged to file named cuda_inverse_opt_2048.txt.

real	0m6.499s
user	0m6.178s
sys	0m0.254s


In [ ]:
sizes = [32,64,128,256,512,1024,2048]
for elem in sizes:
    print(f"Running cuda_optimize for {elem} elements")
    !time ./cuda_optimize < ./ref/{elem}.txt

Running cuda_optimize for 32 elements
Result matrix logged to file named cuda_inverse_opt_32.txt.

real	0m0.556s
user	0m0.092s
sys	0m0.254s
Running cuda_optimize for 64 elements
Result matrix logged to file named cuda_inverse_opt_64.txt.

real	0m0.289s
user	0m0.024s
sys	0m0.214s
Running cuda_optimize for 128 elements
Result matrix logged to file named cuda_inverse_opt_128.txt.

real	0m0.292s
user	0m0.038s
sys	0m0.210s
Running cuda_optimize for 256 elements
Result matrix logged to file named cuda_inverse_opt_256.txt.

real	0m0.274s
user	0m0.074s
sys	0m0.191s
Running cuda_optimize for 512 elements
Result matrix logged to file named cuda_inverse_opt_512.txt.

real	0m0.430s
user	0m0.227s
sys	0m0.197s
Running cuda_optimize for 1024 elements
Result matrix logged to file named cuda_inverse_opt_1024.txt.

real	0m1.258s
user	0m1.041s
sys	0m0.213s
Running cuda_optimize for 2048 elements
Result matrix logged to file named cuda_inverse_opt_2048.txt.

real	0m6.069s
user	0m5.772s
sys	0m0.286s


# Compare W/ Serial

In [ ]:
def read_matrix_from_file(file_path):
    matrix = []
    with open(file_path, 'r') as file:
        N = int(file.readline())

        for _ in range(N):
            row = list(map(float, file.readline().split()))
            matrix.append(row)
    return matrix

def compare_matrix(mtx1, mtx2, eps=1e-4):
    if len(mtx1) != len(mtx2) or len(mtx1[0]) != len(mtx2[0]):
        return False
    flag = True

    for i in range(len(mtx1)):
        for j in range(len(mtx1[0])):
            if abs(mtx1[i][j] - mtx2[i][j]) > eps:
                print(f"Mismatch in {i}: {mtx1[i][j]}, {j}: {mtx2[i][j]} ")
                flag = False
    return flag

sizes = [32,64,128,256,512,1024,2048]
for elem in sizes:
    file_path_cuda = f"./cuda_inverse_opt_{elem}.txt"
    file_path_serial = f"./ref/ser_{elem}.txt"
    matrix_cuda = read_matrix_from_file(file_path_cuda)
    matrix_serial = read_matrix_from_file(file_path_serial)

    if compare_matrix(matrix_cuda, matrix_serial):
        print(f"For {elem} elements: Matrices are approximately equal.")
    else:
        print(f"For {elem} elements: Matrices are not equal.")

For 32 elements: Matrices are approximately equal.
For 64 elements: Matrices are approximately equal.
For 128 elements: Matrices are approximately equal.
For 256 elements: Matrices are approximately equal.
For 512 elements: Matrices are approximately equal.
For 1024 elements: Matrices are approximately equal.
For 2048 elements: Matrices are approximately equal.
